<a href="https://colab.research.google.com/github/gulshanarya/Amazon-Fine-Food-Reviews/blob/main/Amazon%20Fine%20Food%20Reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Downloading data from kaggle:

In [ ]:
!mkdir ~/.kaggle/

In [ ]:
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
!ls -a ~/.kaggle/

.  ..  kaggle.json


In [ ]:
#downloading reviews dataset
!kaggle datasets download snap/amazon-fine-food-reviews -f Reviews.csv

 93% 106M/115M [00:00<00:00, 116MB/s] 
100% 115M/115M [00:01<00:00, 118MB/s]


#Preprocessing Data

In [7]:
#First importing the library used in this project.
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
import numpy as np
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from tqdm import tqdm
from sklearn.preprocessing import normalize
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from bs4 import BeautifulSoup
import scipy
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
import tensorflow as tf
import os
import re
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from gensim.models import KeyedVectors
import os
from collections import Counter
from sklearn.metrics import f1_score
import pandas as pd
import matplotlib.pyplot as plt
import tqdm,bs4,gensim

In [9]:
!python --version

Python 3.7.12


In [17]:
#saving the used version for compatibility.
with open("/content/drive/MyDrive/applied ai/myself work/amazon fine food reviews/requirements.txt", "w") as f:
    f.write(f"""
It is preferred to directly use it in google colab.
Libraries and their versions:
Python 3.7.12
pandas {pd.__version__}
sklearn {sklearn.__version__}
numpy {np.__version__}
nltk {nltk.__version__}
tqdm {tqdm.__version__}
bs4 {bs4.__version__}
scipy {scipy.__version__}
joblib {joblib.__version__}
tensorflow {tf.__version__}
re {re.__version__}
gensim {gensim.__version__}
""")

In [18]:
with open("/content/drive/MyDrive/applied ai/myself work/amazon fine food reviews/requirements.txt", "r") as f:
    print(f.read(), sep="\n")


It is preferred to directly use it in google colab.
Libraries and their versions:
Python 3.7.12
pandas 1.1.5
sklearn 1.0.1
numpy 1.19.5
nltk 3.2.5
tqdm 4.62.3
bs4 4.6.3
scipy 1.4.1
joblib 1.1.0
tensorflow 2.7.0
re 2.2.1
gensim 3.6.0



In [ ]:
data = pd.read_csv("/content/Reviews.csv.zip")

In [ ]:
data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


Data Source: https://www.kaggle.com/snap/amazon-fine-food-reviews

**Objective:**<br>
Given a review determine weather review is +ve (rating 4 or 5) or -ve(rating 1 or 2)


<h2>Exploratory data analysis</h2>

In [ ]:
#dropping all the rows with score = 3
data = data[data["Score"] != 3]

<h3>

<h3>Data deduplication</h3>

First the same user at the same time can't review with the same text on different products but looking below we can say this is the case since look at UserId '#oc-R11D9D7SHXIJB9' below:

In [ ]:
data[data.duplicated(["UserId", "Time", "Text"])].sort_values(["UserId", "Time"]).head(5)

Actually, it's not the error by amazon but instead amazon adding the same review on the same product but with some different flavours and with different ProductId.

In [ ]:
#dropping duplicates
new_data = data.drop_duplicates(["UserId", "ProfileName", "Time", "Text"])

In [ ]:
#Checking to see how much % of data still remains
(new_data['Id'].size*1.0)/(data['Id'].size*1.0)*100

Also HelpfulnessNumerator can't be greater than HelpfulnessDenominator and so removing those rows:

In [ ]:
new_data = new_data[new_data.HelpfulnessNumerator<=new_data.HelpfulnessDenominator]

In [ ]:
new_data["Y"] = (new_data["Score"]>3)*1

**Becoz of computing resource limitation dropping rows from class 1 since class 1 is huge than class 0:**

In [ ]:
#source https://stackoverflow.com/a/54791022/11437685

#making our dataset balanced by dropping rows from class 1(becoz of resource limitation)
def sampling_k_elements(group, k=57110):
    if len(group) < k:
        return group
    return group.sample(k, random_state=42)

balanced_data = new_data.groupby('Y').apply(sampling_k_elements).reset_index(drop=True)

In [ ]:
X = balanced_data["Text"].apply(lambda sent: sent.lower())
Y = balanced_data["Y"]

In [ ]:
#balanced dataset
print(X.shape, Y.value_counts(), sep="\n")

In [ ]:
#balaced dataset
(Y.value_counts()/Y.shape[0]).plot(kind='bar')

##Text preprocessing:

In [ ]:
# https://stackoverflow.com/a/47091490/4084039

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [ ]:
nltk.download('stopwords')

In [ ]:
stpwrds = stopwords.words('english')

#removing some of the words from the stopwords since here these words are very useful for our task
stpwrds.remove("not")
stpwrds.remove("nor")
stpwrds.remove("no")

In [ ]:
#removing all the urls since not useful in classificatioin
X = X.apply(lambda sent: re.sub(r"http\S+", "", sent))

#used for just getting text and removing all the html tags
X = X.apply(lambda sent: BeautifulSoup(sent, 'lxml').get_text())

#Decontracting
X = X.apply(decontracted)

#removing words with numbers and then removing extra spaces at start and end
X = X.apply(lambda sent: re.sub("\S*\d\S*", "", sent).strip())

X = X.apply(lambda sent: ' '.join(w for w in sent.split() if w not in stpwrds))

**Apply stemming:**

Here we will do stemming where the inflected words will get reduced to their root form i.e. word stem:

In [ ]:
nltk.download('punkt')

In [ ]:
porter = PorterStemmer()

In [ ]:
def stemming(sent):
    #getting actual word from the sent
    tokens = word_tokenize(sent)
    return " ".join(porter.stem(w) for w in tokens)

In [ ]:
X = X.apply(stemming)

In [ ]:
#printing some random text
print(X[10])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=42, stratify=Y)

**Saving preprocessed data:**

In [ ]:
# X_train.to_csv("/content/drive/MyDrive/applied ai/myself work/amazon fine food reviews/preprocessed_data/X_train.csv", index=False)
# X_test.to_csv("/content/drive/MyDrive/applied ai/myself work/amazon fine food reviews/preprocessed_data/X_test.csv", index=False)
# y_train.to_csv("/content/drive/MyDrive/applied ai/myself work/amazon fine food reviews/preprocessed_data/y_train.csv", index=False)
# y_test.to_csv("/content/drive/MyDrive/applied ai/myself work/amazon fine food reviews/preprocessed_data/y_test.csv", index=False)

**Load preprocessed data:**

In [ ]:
# X_train1 = pd.read_csv("/content/drive/MyDrive/applied ai/myself work/amazon fine food reviews/preprocessed_data/X_train.csv")["Text"]
# X_test1 = pd.read_csv("/content/drive/MyDrive/applied ai/myself work/amazon fine food reviews/preprocessed_data/X_test.csv")["Text"]
y_train = np.array(pd.read_csv("/content/drive/MyDrive/applied ai/myself work/amazon fine food reviews/preprocessed_data/y_train.csv")["Y"])
y_test = np.array(pd.read_csv("/content/drive/MyDrive/applied ai/myself work/amazon fine food reviews/preprocessed_data/y_test.csv")["Y"])

## Featurization:

### BOW vectorization:

In [ ]:
bow = CountVectorizer(ngram_range=(1,2), min_df=10, max_features=5000)
X_train_bow = bow.fit_transform(X_train)

In [ ]:
X_test_bow = bow.transform(X_test)

In [ ]:
#sparse matrix
X_train_bow

In [ ]:
#all the top selected features
print(bow.get_feature_names_out())

**Saving preprocessed data:**

In [ ]:
# scipy.sparse.save_npz('/content/drive/MyDrive/applied ai/myself work/amazon fine food reviews/preprocessed_data/X_train_bow.npz', X_train_bow)
# scipy.sparse.save_npz('/content/drive/MyDrive/applied ai/myself work/amazon fine food reviews/preprocessed_data/X_test_bow.npz', X_test_bow)

**Loading preprocessed data:**

In [ ]:
X_train_bow = scipy.sparse.load_npz('/content/drive/MyDrive/applied ai/myself work/amazon fine food reviews/preprocessed_data/X_train_bow.npz')
X_test_bow = scipy.sparse.load_npz('/content/drive/MyDrive/applied ai/myself work/amazon fine food reviews/preprocessed_data/X_test_bow.npz')

**Feature Normalisation**

In [ ]:
#using MaxAbsScaler becoz it works with the sparse and we have only +ve data and so similar to MinMaxScaler
scaler_bow = MaxAbsScaler().fit(X_train_bow)

In [ ]:
X_train_bow = scaler_bow.transform(X_train_bow)

In [ ]:
X_test_bow = scaler_bow.transform(X_test_bow)

**Saving bow and scaler_bow:**

In [ ]:
#we will use it later for avoiding data leakage
with open("/content/drive/MyDrive/applied ai/myself work/amazon fine food reviews/saved_vectorizer/bow_scaler.pkl", 'wb') as f:
    pickle.dump((bow, scaler_bow), f)

###TFIDF Featurization:

In [ ]:
tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=10, max_features=10000)
X_train_tfidf = tfidf.fit_transform(X_train)

In [ ]:
X_test_tfidf = tfidf.transform(X_test)

In [ ]:
#a sparse matrix
X_train_tfidf

In [ ]:
#all the top selected features
print(tfidf.get_feature_names_out())

**Saving preprocessed data:**

In [ ]:
# scipy.sparse.save_npz('/content/drive/MyDrive/applied ai/myself work/amazon fine food reviews/preprocessed_data/X_train_tfidf.npz', X_train_tfidf)
# scipy.sparse.save_npz('/content/drive/MyDrive/applied ai/myself work/amazon fine food reviews/preprocessed_data/X_test_tfidf.npz', X_test_tfidf)

**Loading preprocessed data:**

In [ ]:
X_train_tfidf = scipy.sparse.load_npz('/content/drive/MyDrive/applied ai/myself work/amazon fine food reviews/preprocessed_data/X_train_tfidf.npz')
X_test_tfidf = scipy.sparse.load_npz('/content/drive/MyDrive/applied ai/myself work/amazon fine food reviews/preprocessed_data/X_test_tfidf.npz')

**Feature Normalisation**

In [ ]:
#using MaxAbsScaler becoz it works with the sparse and we have only +ve data and so similar to MinMaxScaler
scaler_tfidf = MaxAbsScaler().fit(X_train_tfidf)

In [ ]:
X_train_tfidf = scaler_tfidf.transform(X_train_tfidf)

In [ ]:
X_test_tfidf = scaler_tfidf.transform(X_test_tfidf)

### Word2Vec Vectorization:

In [ ]:
# !pip install gdown
#downloading a pretrained model by google
# !gdown --id "https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM"

In [ ]:

#using a pretrained model by google
if os.path.isfile('/content/drive/MyDrive/applied ai/myself work/amazon fine food reviews/GoogleNews-vectors-negative300.bin.gz'):
    w2v_model=KeyedVectors.load_word2vec_format('/content/drive/MyDrive/applied ai/myself work/amazon fine food reviews/GoogleNews-vectors-negative300.bin.gz', binary=True)

### Avg Word2Vec

In [ ]:
#All the learned vocabulary 
# vocab = w2v_model.key_to_index
vocab = w2v_model.vocab

In [ ]:
#func for avgW2V
def avgW2vec(sent):
    lst_wrds = sent.split(" ")
    #store avgw2v of size 300
    avg_w2v = np.zeros(300,dtype='float64')
    for w in lst_wrds:
        if w in vocab:
            avg_w2v += w2v_model[w]
    n = len(lst_wrds)
    if n != 0: avg_w2v /= n
    else: raise Exception(f"Here len(query) can't be {n}")
    return avg_w2v

#avgW2vec on each sent in X_train
X_train_avgw2v = X_train.apply(avgW2vec)

In [ ]:
#Having some problems of dtype and dims, when directly converting to np array and so first series to list and then list to np array
X_train_avgw2v = X_train_avgw2v.to_list()
X_train_avgw2v = np.array(X_train_avgw2v)

In [ ]:
X_test_avgw2v = X_test.apply(avgW2vec)

In [ ]:
#Having some problems of dtype and dims, when directly converting to np array and so first series to list and then list to np array
X_test_avgw2v = X_test_avgw2v.to_list()
X_test_avgw2v = np.array(X_test_avgw2v)

**Saving preprocessed data:**

In [ ]:
# with open('/content/drive/MyDrive/applied ai/myself work/amazon fine food reviews/preprocessed_data/X_train_avgw2v.npz', "wb") as f:
#     np.save(f, X_train_avgw2v)

# with open('/content/drive/MyDrive/applied ai/myself work/amazon fine food reviews/preprocessed_data/X_test_avgw2v.npz', "wb") as f:
#     np.save(f, X_test_avgw2v)

**Loading preprocessed data:**

In [ ]:
with open('/content/drive/MyDrive/applied ai/myself work/amazon fine food reviews/preprocessed_data/X_train_avgw2v.npz', "rb") as f:
    X_train_avgw2v = np.load(f)

with open('/content/drive/MyDrive/applied ai/myself work/amazon fine food reviews/preprocessed_data/X_test_avgw2v.npz', "rb") as f:
    X_test_avgw2v = np.load(f)

**Feature Normalisation**

In [ ]:
scaler_avgw2v = MinMaxScaler().fit(X_train_avgw2v)

In [ ]:
X_train_avgw2v = scaler_avgw2v.transform(X_train_avgw2v)

In [ ]:
X_test_avgw2v = scaler_avgw2v.transform(X_test_avgw2v)

### TFIDF-W2V

In [ ]:
#getting all the feature names used in tfidf for tfidf-w2v
idf_dict = dict(zip(tfidf.get_feature_names_out(), list(tfidf.idf_)))

In [ ]:
#func for avgW2V
def tfidfW2V(sent):
    #storing all tfidf of each word for computing total tfidf of a sent
    tfidf_vec = list()
    
    #splitting and counting # of each words in sent
    lst_wrds = sent.split(" ")
    cnt_wrds = Counter(lst_wrds)
    #n for computing the tf
    n = len(lst_wrds)
    
    #store avgw2v of size 300
    tfidf_w2v = np.zeros(300,dtype='float64')
    
    for w in cnt_wrds:
        if w in vocab:
            try:
                idf = idf_dict[w]
                tf = cnt_wrds[w]/n
                tfidf = tf*idf
                
                #adding for computing total tfidf
                tfidf_vec.append(tfidf)

            except:
                continue
                
            tfidf_w2v += w2v_model[w]*tfidf
            
    total_tfidf = np.sum(tfidf_vec)
    if total_tfidf != 0: tfidf_w2v /= total_tfidf
#     else: raise Exception(f"Here total_tfidf can't be {total_tfidf} for sent at index {sent_i}")
    return tfidf_w2v

#tfidfW2vec on each sent in X_train
X_train_tfidf_w2v = X_train.apply(tfidfW2V)

In [ ]:
#Having some problems of dtype and dims, when directly converting to np array and so first series to list and then list to np array
X_train_tfidf_w2v = X_train_tfidf_w2v.to_list()
X_train_tfidf_w2v = np.array(X_train_tfidf_w2v)

In [ ]:
#bug in colledting at X_test.iloc[12478]

In [ ]:
#Having some problems of dtype and dims, when directly converting to np array and so first series to list and then list to np array
X_test_tfidf_w2v = X_test.apply(tfidfW2V)

X_test_tfidf_w2v = X_test_tfidf_w2v.to_list()
X_test_tfidf_w2v = np.array(X_test_tfidf_w2v)

**Saving preprocessed data:**

In [ ]:
# with open('/content/drive/MyDrive/applied ai/myself work/amazon fine food reviews/preprocessed_data/X_train_tfidf_w2v.npz', "wb") as f:
#     np.save(f, X_train_tfidf_w2v)

# with open('/content/drive/MyDrive/applied ai/myself work/amazon fine food reviews/preprocessed_data/X_test_tfidf_w2v.npz', "wb") as f:
#     np.save(f, X_test_tfidf_w2v)

**Loading preprocessed data:**

In [ ]:
with open('/content/drive/MyDrive/applied ai/myself work/amazon fine food reviews/preprocessed_data/X_train_tfidf_w2v.npz', "rb") as f:
    X_train_tfidfw2v = np.load(f)

with open('/content/drive/MyDrive/applied ai/myself work/amazon fine food reviews/preprocessed_data/X_test_avgw2v.npz', "rb") as f:
    X_test_tfidfw2v = np.load(f)

**Feature Normalisation**

In [ ]:
scaler_tfidfw2v = MinMaxScaler().fit(X_train_tfidfw2v)

In [ ]:
X_train_tfidfw2v = scaler_tfidfw2v.transform(X_train_tfidfw2v)

In [ ]:
X_test_tfidfw2v = scaler_tfidfw2v.transform(X_test_tfidfw2v)

# ML models using bow

## Applying MNB

In [ ]:
mnb = MultinomialNB()

parameters = {'alpha': [0.0001, 0.01, 0.1, 1, 10, 100]}
mnb_tune = GridSearchCV(mnb, parameters, n_jobs=-1, verbose=2)
mnb_tune.fit(X_train_bow, y_train)

In [ ]:
(mnb_tune.cv_results_)

In [ ]:
#mnb with best hyperparameter
best_mnb = mnb_tune.best_estimator_

### Performance of mnb model

In [ ]:
#train predicted probability
train_proba = best_mnb.predict_proba(X_train_bow)

#test predicted probability
test_proba = best_mnb.predict_proba(X_test_bow)

In [ ]:
#train predicted
y_train_predict = best_mnb.predict(X_train_bow)

#test predicted
y_test_predict = best_mnb.predict(X_test_bow)

In [ ]:
#accuracy score on the train data
accuracy_score(y_train, y_train_predict)

In [ ]:
#accuracy score on the test data
accuracy_score(y_test, y_test_predict)

In [ ]:
#confusion_matrix on train data

#rows are actural label in ascending order i.e. 0 then 1 and cols are predicted labels
confusion_matrix(y_train, y_train_predict, normalize='true')

In [ ]:
#confusion_matrix on test data
#rows are actual label in ascending order i.e. 0 then 1 and cols are predicted labels
confusion_matrix(y_test, y_test_predict, normalize='true')

In [ ]:

#f1_score on the train data
print(f1_score(y_train, y_train_predict))

#f1_score on the test data
print(f1_score(y_test, y_test_predict))

**Observation: Performance is good. From confusion matrix, we can say approx 11% of +ve data predicted as -ve and approx 13% of -ve data predicted as +ve on the test data.**

**Saving trained model:**

In [ ]:
joblib.dump(best_mnb, "/content/drive/MyDrive/applied ai/myself work/amazon fine food reviews/saved_models/best_mnb_bow.sav")

**Loading trained data:**

In [ ]:
best_mnb = joblib.load("/content/drive/MyDrive/applied ai/myself work/amazon fine food reviews/saved_models/best_mnb_bow.sav")

##Applying Logistic regression

In [ ]:
#max_iter=1000 becoz of high value of C(for small learning rate) below like 100 for converging to minm
lgr = LogisticRegression(max_iter=1000)

parameters = {'C':[0.0001, 0.001, 0.01, 0.1, 1, 10, 100]}
lgr_tune = GridSearchCV(lgr, parameters, n_jobs=-1, verbose=2)
lgr_tune.fit(X_train_bow, y_train)

In [ ]:
(lgr_tune.cv_results_)

In [ ]:
#lgr with best hyperparameter
best_lgr = lgr_tune.best_estimator_

### Performance of lgr model

In [ ]:
#train predicted probability
train_proba = best_lgr.predict_proba(X_train_bow)

#test predicted probability
test_proba = best_lgr.predict_proba(X_test_bow)

In [ ]:
#train predicted
y_train_predict = best_lgr.predict(X_train_bow)

#test predicted
y_test_predict = best_lgr.predict(X_test_bow)

In [ ]:
#accuracy score on the train data
accuracy_score(y_train, y_train_predict)

In [ ]:
#accuracy score on the test data
accuracy_score(y_test, y_test_predict)

In [ ]:
#confusion_matrix on train data

#rows are actual label in ascending order i.e. 0 then 1 and cols are predicted labels
confusion_matrix(y_train, y_train_predict, normalize='true')

In [ ]:
#confusion_matrix on test data
#rows are actual label in ascending order i.e. 0 then 1 and cols are predicted labels
confusion_matrix(y_test, y_test_predict, normalize='true')

In [ ]:

#f1_score on the train data
print(f1_score(y_train, y_train_predict))

#f1_score on the test data
print(f1_score(y_test, y_test_predict))

**Observation: Performance is fairly better than the knn. Also by looking at the confusion matrix, we can say performance is not bias towards any class**

**Saving trained model:**

In [ ]:
# joblib.dump(best_lgr, "/content/drive/MyDrive/applied ai/myself work/amazon fine food reviews/saved_models/best_lgr.sav")

**Loading trained data:**

In [ ]:
best_lgr = joblib.load("/content/drive/MyDrive/applied ai/myself work/amazon fine food reviews/saved_models/best_lgr.sav")

## Applying DNN

In [ ]:
#splitting train data into validation and the train set
X_train_bow = X_train_bow.toarray()
X_test_bow = X_test_bow.toarray()
X_valid_bow, X_train_bow = X_train_bow[:10000], X_train_bow[10000:]
y_valid, y_train = y_train[:10000], y_train[10000:]

In [ ]:
#applying simple NN with dropouts
dnn = tf.keras.models.Sequential([
  tf.keras.layers.InputLayer(input_shape=(X_train_bow.shape[1],)),
  tf.keras.layers.Dense(128, activation='elu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(128, activation='elu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
# compile the keras model
dnn.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [ ]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

In [ ]:
history = dnn.fit(X_train_bow, y_train, epochs=30, validation_data=(X_valid_bow, y_valid), callbacks=[early_stopping_cb])

### Performance of DNN model

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1) # set the vertical range to [0-1]
plt.show()

In [ ]:
dnn.evaluate(X_train_bow, y_train, batch_size=128)

In [ ]:
dnn.evaluate(X_test_bow, y_test, batch_size=128)

In [ ]:
#train predicted probability
train_proba = dnn.predict(X_train_bow)

#test predicted probability
test_proba = dnn.predict(X_test_bow)

In [ ]:
#predicted classes on train data
y_train_predict = train_proba >0.5

#predicted classes on test data
y_test_predict = test_proba>0.5

In [ ]:
#accuracy score on train data
accuracy_score(y_train,y_train_predict)

In [ ]:
#accuracy score on test data
accuracy_score(y_test,y_test_predict)

In [ ]:
#confusion_matrix on train data

#rows are actual label in ascending order i.e. 0 then 1 and cols are predicted labels
confusion_matrix(y_train, y_train_predict, normalize='true')

In [ ]:
#confusion_matrix on test data
#rows are actual label in ascending order i.e. 0 then 1 and cols are predicted labels
confusion_matrix(y_test, y_test_predict, normalize='true')

In [ ]:
from sklearn.metrics import f1_score

#f1_score on the train data
print(f1_score(y_train, y_train_predict))

#f1_score on the test data
print(f1_score(y_test, y_test_predict))

**Observation: Here we are overfittng little bit, certainly we need to tune our model hyperparameters or apply regularization for getting the good performance on the test set as well.**

In [ ]:
#model saving path
model_version = "0001"
model_path = "/content/drive/MyDrive/applied ai/myself work/amazon fine food reviews/saved_models/dnn_bow"

**Saving trained model:**

In [ ]:
import os

In [ ]:
# model_path = os.path.join(model_path, model_version)
# dnn.save(model_path)

**Loading trained data:**

In [ ]:
new_model = tf.keras.models.load_model(model_path)

In [ ]:
new_model.summary()

# ML models using TFIDF

## Applying MNB

In [ ]:
mnb = MultinomialNB()

parameters = {'alpha': [0.0001, 0.01, 0.1, 1, 10, 100]}
mnb_tune = GridSearchCV(mnb, parameters, n_jobs=-1, verbose=2)
mnb_tune.fit(X_train_tfidf, y_train)

In [ ]:
(mnb_tune.cv_results_)

In [ ]:
#mnb with best hyperparameter
best_mnb = mnb_tune.best_estimator_

### Performance of mnb model

In [ ]:
#train predicted probability
train_proba = best_mnb.predict_proba(X_train_tfidf)

#test predicted probability
test_proba = best_mnb.predict_proba(X_test_tfidf)

In [ ]:
#train predicted
y_train_predict = best_mnb.predict(X_train_tfidf)

#test predicted
y_test_predict = best_mnb.predict(X_test_tfidf)

In [ ]:
#accuracy score on the train data
accuracy_score(y_train, y_train_predict)

In [ ]:
#accuracy score on the test data
accuracy_score(y_test, y_test_predict)

In [ ]:
#confusion_matrix on train data

#rows are actural label in ascending order i.e. 0 then 1 and cols are predicted labels
confusion_matrix(y_train, y_train_predict, normalize='true')

In [ ]:
#confusion_matrix on test data
#rows are actual label in ascending order i.e. 0 then 1 and cols are predicted labels
confusion_matrix(y_test, y_test_predict, normalize='true')

In [ ]:

#f1_score on the train data
print(f1_score(y_train, y_train_predict))

#f1_score on the test data
print(f1_score(y_test, y_test_predict))

**Observation: Performance is good. From confusion matrix, we can say approx 11% of +ve data predicted as -ve and approx 11% of -ve data predicted as +ve on the test data.**

**Saving trained model:**

In [ ]:
joblib.dump(best_mnb, "/content/drive/MyDrive/applied ai/myself work/amazon fine food reviews/saved_models/best_mnb_tfidf.sav")

**Loading trained data:**

In [ ]:
best_mnb = joblib.load("/content/drive/MyDrive/applied ai/myself work/amazon fine food reviews/saved_models/best_mnb_tfidf.sav")

##Applying Logistic regression

In [ ]:
#max_iter=1000 becoz of high value of C(for small learning rate) below like 100 for converging to minm
lgr = LogisticRegression(max_iter=1000)

parameters = {'C':[0.0001, 0.001, 0.01, 0.1, 1, 10, 100]}
lgr_tune = GridSearchCV(lgr, parameters, n_jobs=-1, verbose=2)
lgr_tune.fit(X_train_tfidf, y_train)

In [ ]:
(lgr_tune.cv_results_)

In [ ]:
#lgr with best hyperparameter
best_lgr = lgr_tune.best_estimator_

### Performance of lgr model

In [ ]:
#train predicted probability
train_proba = best_lgr.predict_proba(X_train_tfidf)

#test predicted probability
test_proba = best_lgr.predict_proba(X_test_tfidf)

In [ ]:
#train predicted
y_train_predict = best_lgr.predict(X_train_tfidf)

#test predicted
y_test_predict = best_lgr.predict(X_test_tfidf)

In [ ]:
#accuracy score on the train data
accuracy_score(y_train, y_train_predict)

In [ ]:
#accuracy score on the test data
accuracy_score(y_test, y_test_predict)

In [ ]:
#confusion_matrix on train data

#rows are actual label in ascending order i.e. 0 then 1 and cols are predicted labels
confusion_matrix(y_train, y_train_predict, normalize='true')

In [ ]:
#confusion_matrix on test data
#rows are actual label in ascending order i.e. 0 then 1 and cols are predicted labels
confusion_matrix(y_test, y_test_predict, normalize='true')

In [ ]:
from sklearn.metrics import f1_score

#f1_score on the train data
print(f1_score(y_train, y_train_predict))

#f1_score on the test data
print(f1_score(y_test, y_test_predict))

**Observation: Performance is fairly better than the knn.**

**Saving trained model:**

In [ ]:
# joblib.dump(best_lgr, "/content/drive/MyDrive/applied ai/myself work/amazon fine food reviews/saved_models/best_lgr_tfidf.sav")

**Loading trained data:**

In [ ]:
best_lgr = joblib.load("/content/drive/MyDrive/applied ai/myself work/amazon fine food reviews/saved_models/best_lgr_tfidf.sav")

## Applying DNN

In [ ]:
#splitting train data into validation and the train set
X_train_tfidf.sort_indices()
X_test_tfidf.sort_indices()
X_valid_tfidf, X_train_tfidf = X_train_tfidf[:10000], X_train_tfidf[10000:]
y_valid, y_train = y_train[:10000], y_train[10000:]

In [ ]:
#applying simple NN with dropouts
dnn = tf.keras.models.Sequential([
  tf.keras.layers.InputLayer(input_shape=(X_train_tfidf.shape[1],), sparse=True),
  tf.keras.layers.Dense(128, activation='elu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(128, activation='elu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
# compile the keras model
dnn.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [ ]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

In [ ]:
history = dnn.fit(X_train_tfidf, y_train, epochs=30, validation_data=(X_valid_tfidf, y_valid), callbacks=[early_stopping_cb])

### Performance of DNN model

In [ ]:
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1) # set the vertical range to [0-1]
plt.show()

In [ ]:
dnn.evaluate(X_train_tfidf, y_train, batch_size=128)

In [ ]:
dnn.evaluate(X_test_tfidf, y_test, batch_size=128)

In [ ]:
#train predicted probability
train_proba = dnn.predict(X_train_tfidf)

#test predicted probability
test_proba = new_model.predict(X_test_tfidf)

In [ ]:
#predicted classes on train data
y_train_predict = train_proba >0.5

#predicted classes on test data
y_test_predict = test_proba>0.5

In [ ]:
#accuracy score on train data
accuracy_score(y_train,y_train_predict)

In [ ]:
#accuracy score on test data
accuracy_score(y_test,y_test_predict)

In [ ]:
#confusion_matrix on train data

#rows are actual label in ascending order i.e. 0 then 1 and cols are predicted labels
confusion_matrix(y_train, y_train_predict, normalize='true')

In [ ]:
#confusion_matrix on test data
#rows are actual label in ascending order i.e. 0 then 1 and cols are predicted labels
confusion_matrix(y_test, y_test_predict, normalize='true')

In [ ]:

#f1_score on the train data
print(f1_score(y_train, y_train_predict))

#f1_score on the test data
print(f1_score(y_test, y_test_predict))

**Observation: Here we are overfittng little bit, certainly we need to tune our model hyperparameters or apply regularization for getting the good performance on the test set as well.**

In [ ]:
#model saving path
model_version = "0001"
model_path = "/content/drive/MyDrive/applied ai/myself work/amazon fine food reviews/saved_models/dnn_tfidf"
model_path = os.path.join(model_path, model_version)

**Saving trained model:**

In [ ]:
# dnn.save(model_path)

**Loading trained data:**

In [ ]:
new_model = tf.keras.models.load_model(model_path)

In [ ]:
new_model.summary()

#ML models using avgW2V

## Applying MNB

In [ ]:
mnb = MultinomialNB()

parameters = {'alpha': [0.0001, 0.01, 0.1, 1, 10, 100]}
mnb_tune = GridSearchCV(mnb, parameters, n_jobs=-1, verbose=2)
mnb_tune.fit(X_train_avgw2v, y_train)

In [ ]:
(mnb_tune.cv_results_)

In [ ]:
#mnb with best hyperparameter
best_mnb = mnb_tune.best_estimator_

### Performance of mnb model

In [ ]:
#train predicted probability
train_proba = best_mnb.predict_proba(X_train_avgw2v)

#test predicted probability
test_proba = best_mnb.predict_proba(X_test_avgw2v)

In [ ]:
#train predicted
y_train_predict = best_mnb.predict(X_train_avgw2v)

#test predicted
y_test_predict = best_mnb.predict(X_test_avgw2v)

In [ ]:
#accuracy score on the train data
accuracy_score(y_train, y_train_predict)

In [ ]:
#accuracy score on the test data
accuracy_score(y_test, y_test_predict)

In [ ]:
#confusion_matrix on train data

#rows are actural label in ascending order i.e. 0 then 1 and cols are predicted labels
confusion_matrix(y_train, y_train_predict, normalize='true')

In [ ]:
#confusion_matrix on test data
#rows are actual label in ascending order i.e. 0 then 1 and cols are predicted labels
confusion_matrix(y_test, y_test_predict, normalize='true')

In [ ]:

#f1_score on the train data
print(f1_score(y_train, y_train_predict))

#f1_score on the test data
print(f1_score(y_test, y_test_predict))

**Observation: Performance is not good. From confusion matrix, we can say approx 33% of +ve data predicted as -ve and approx 27% of -ve data predicted as +ve on the test data.**

**Saving trained model:**

In [ ]:
joblib.dump(best_mnb, "/content/drive/MyDrive/applied ai/myself work/amazon fine food reviews/saved_models/best_mnb_avgw2v.sav")

**Loading trained data:**

In [ ]:
best_mnb = joblib.load("/content/drive/MyDrive/applied ai/myself work/amazon fine food reviews/saved_models/best_mnb_avgw2v.sav")

##Applying Logistic regression

In [ ]:
#max_iter=1000 becoz of high value of C(for small learning rate) below like 100 for converging to minm
lgr = LogisticRegression(max_iter=1000)

parameters = {'C':[0.0001, 0.001, 0.01, 0.1, 1, 10, 100]}
lgr_tune = GridSearchCV(lgr, parameters, n_jobs=-1, verbose=2)
lgr_tune.fit(X_train_avgw2v, y_train)

In [ ]:
(lgr_tune.cv_results_)

In [ ]:
#lgr with best hyperparameter
best_lgr = lgr_tune.best_estimator_

### Performance of lgr model

In [ ]:
#train predicted probability
train_proba = best_lgr.predict_proba(X_train_avgw2v)

#test predicted probability
test_proba = best_lgr.predict_proba(X_test_avgw2v)

In [ ]:
#train predicted
y_train_predict = best_lgr.predict(X_train_avgw2v)

#test predicted
y_test_predict = best_lgr.predict(X_test_avgw2v)

In [ ]:
#accuracy score on the train data
accuracy_score(y_train, y_train_predict)

In [ ]:
#accuracy score on the test data
accuracy_score(y_test, y_test_predict)

In [ ]:
#confusion_matrix on train data

#rows are actual label in ascending order i.e. 0 then 1 and cols are predicted labels
confusion_matrix(y_train, y_train_predict, normalize='true')

In [ ]:
#confusion_matrix on test data
#rows are actual label in ascending order i.e. 0 then 1 and cols are predicted labels
confusion_matrix(y_test, y_test_predict, normalize='true')

In [ ]:

#f1_score on the train data
print(f1_score(y_train, y_train_predict))

#f1_score on the test data
print(f1_score(y_test, y_test_predict))

**Observation: Performance is better than the knn but avgw2v performing badly than the tfidf and bow**

**Saving trained model:**

In [ ]:
# joblib.dump(best_lgr, "/content/drive/MyDrive/applied ai/myself work/amazon fine food reviews/saved_models/best_lgr_avgw2v.sav")

**Loading trained data:**

In [ ]:
best_lgr = joblib.load("/content/drive/MyDrive/applied ai/myself work/amazon fine food reviews/saved_models/best_lgr_avgw2v.sav")

## Applying DNN

In [ ]:
#splitting train data into validation and the train set
X_valid_avgw2v, X_train_avgw2v = X_train_avgw2v[:10000], X_train_avgw2v[10000:]
y_valid, y_train = y_train[:10000], y_train[10000:]

In [ ]:
#applying simple NN with dropouts
dnn = tf.keras.models.Sequential([
  tf.keras.layers.InputLayer(input_shape=(X_train_avgw2v.shape[1],)),
  tf.keras.layers.Dense(128, activation='elu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(128, activation='elu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
# compile the keras model
dnn.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [ ]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

In [ ]:
history = dnn.fit(X_train_avgw2v, y_train, epochs=30, validation_data=(X_valid_avgw2v, y_valid), callbacks=[early_stopping_cb])

### Performance of DNN model

In [ ]:

pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1) # set the vertical range to [0-1]
plt.show()

In [ ]:
dnn.evaluate(X_train_avgw2v, y_train, batch_size=128)

In [ ]:
dnn.evaluate(X_test_avgw2v, y_test, batch_size=128)

In [ ]:
#train predicted probability
train_proba = dnn.predict(X_train_avgw2v)

#test predicted probability
test_proba = dnn.predict(X_test_avgw2v)

In [ ]:
#predicted classes on train data
y_train_predict = train_proba >0.5

#predicted classes on test data
y_test_predict = test_proba>0.5

In [ ]:
#accuracy score on train data
accuracy_score(y_train,y_train_predict)

In [ ]:
#accuracy score on test data
accuracy_score(y_test,y_test_predict)

In [ ]:
#confusion_matrix on train data

#rows are actual label in ascending order i.e. 0 then 1 and cols are predicted labels
confusion_matrix(y_train, y_train_predict, normalize='true')

In [ ]:
#confusion_matrix on test data
#rows are actual label in ascending order i.e. 0 then 1 and cols are predicted labels
confusion_matrix(y_test, y_test_predict, normalize='true')

In [ ]:

#f1_score on the train data
print(f1_score(y_train, y_train_predict))

#f1_score on the test data
print(f1_score(y_test, y_test_predict))

**Observation: Look at above confusion matrix: approx 30% of +ve data predicted as -ve and 11% of -ve data predicted as +ve. And also overall performance is not good. It is performing badly than the bow and tfidf vectorization.**

In [ ]:
#model saving path
model_version = "0001"
model_path = "/content/drive/MyDrive/applied ai/myself work/amazon fine food reviews/saved_models/dnn_avgw2v"
model_path = os.path.join(model_path, model_version)

**Saving trained model:**

In [ ]:
# dnn.save(model_path)

**Loading trained data:**

In [ ]:
new_model = tf.keras.models.load_model(model_path)

In [ ]:
new_model.summary()

#ML models using TFIDF-W2V

## Applying MNB

In [ ]:
mnb = MultinomialNB()

parameters = {'alpha': [0.0001, 0.01, 0.1, 1, 10, 100]}
mnb_tune = GridSearchCV(mnb, parameters, n_jobs=-1, verbose=2)
mnb_tune.fit(X_train_tfidfw2v, y_train)

In [ ]:
(mnb_tune.cv_results_)

In [ ]:
#mnb with best hyperparameter
best_mnb = mnb_tune.best_estimator_

### Performance of mnb model

In [ ]:
#train predicted probability
train_proba = best_mnb.predict_proba(X_train_tfidfw2v)

#test predicted probability
test_proba = best_mnb.predict_proba(X_test_tfidfw2v)

In [ ]:
#train predicted
y_train_predict = best_mnb.predict(X_train_tfidfw2v)

#test predicted
y_test_predict = best_mnb.predict(X_test_tfidfw2v)

In [ ]:
#accuracy score on the train data
accuracy_score(y_train, y_train_predict)

In [ ]:
#accuracy score on the test data
accuracy_score(y_test, y_test_predict)

In [ ]:
#confusion_matrix on train data

#rows are actural label in ascending order i.e. 0 then 1 and cols are predicted labels
confusion_matrix(y_train, y_train_predict, normalize='true')

In [ ]:
#confusion_matrix on test data
#rows are actual label in ascending order i.e. 0 then 1 and cols are predicted labels
confusion_matrix(y_test, y_test_predict, normalize='true')

In [ ]:

#f1_score on the train data
print(f1_score(y_train, y_train_predict))

#f1_score on the test data
print(f1_score(y_test, y_test_predict))

**Observation: Our model badly performing. From confusion matrix, we can say approx 57% of +ve data predicted as -ve and only approx 10% of -ve data predicted as +ve on the test data.**

**Saving trained model:**

In [ ]:
joblib.dump(best_mnb, "/content/drive/MyDrive/applied ai/myself work/amazon fine food reviews/saved_models/best_mnb_tfidfw2v.sav")

**Loading trained data:**

In [ ]:
best_mnb = joblib.load("/content/drive/MyDrive/applied ai/myself work/amazon fine food reviews/saved_models/best_mnb_tfidfw2v.sav")

##Applying Logistic regression

In [ ]:
#max_iter=1000 becoz of high value of C(for small learning rate) below like 100 for converging to minm
lgr = LogisticRegression(max_iter=1000)

parameters = {'C':[0.0001, 0.001, 0.01, 0.1, 1, 10, 100]}
lgr_tune = GridSearchCV(lgr, parameters, n_jobs=-1, verbose=2)
lgr_tune.fit(X_train_tfidfw2v, y_train)

In [ ]:
(lgr_tune.cv_results_)

In [ ]:
#lgr with best hyperparameter
best_lgr = lgr_tune.best_estimator_

### Performance of lgr model

In [ ]:
#train predicted probability
train_proba = best_lgr.predict_proba(X_train_tfidfw2v)

#test predicted probability
test_proba = best_lgr.predict_proba(X_test_tfidfw2v)

In [ ]:
#train predicted
y_train_predict = best_lgr.predict(X_train_tfidfw2v)

#test predicted
y_test_predict = best_lgr.predict(X_test_tfidfw2v)

In [ ]:
#accuracy score on the train data
accuracy_score(y_train, y_train_predict)

In [ ]:
#accuracy score on the test data
accuracy_score(y_test, y_test_predict)

In [ ]:
#confusion_matrix on train data

#rows are actual label in ascending order i.e. 0 then 1 and cols are predicted labels
confusion_matrix(y_train, y_train_predict, normalize='true')

In [ ]:
#confusion_matrix on test data
#rows are actual label in ascending order i.e. 0 then 1 and cols are predicted labels
confusion_matrix(y_test, y_test_predict, normalize='true')

In [ ]:

#f1_score on the train data
print(f1_score(y_train, y_train_predict))

#f1_score on the test data
print(f1_score(y_test, y_test_predict))

**Observation: Look at above confusion matrix: approx 21% of +ve data predicted as -ve and 17% of -ve data predicted as +ve. And also overall performance is not very good. It is performing badly than the bow and tfidf vectorization but better than the avgw2v.**

**Saving trained model:**

In [ ]:
# joblib.dump(best_lgr, "/content/drive/MyDrive/applied ai/myself work/amazon fine food reviews/saved_models/best_lgr_tfidfw2v.sav")

**Loading trained data:**

In [ ]:
best_lgr = joblib.load("/content/drive/MyDrive/applied ai/myself work/amazon fine food reviews/saved_models/best_lgr_tfidfw2v.sav")

## Applying DNN

In [ ]:
#splitting train data into validation and the train set
X_valid_tfidfw2v, X_train_tfidfw2v = X_train_tfidfw2v[:10000], X_train_tfidfw2v[10000:]
y_valid, y_train = y_train[:10000], y_train[10000:]

In [ ]:
#applying simple NN with dropouts
dnn = tf.keras.models.Sequential([
  tf.keras.layers.InputLayer(input_shape=(X_train_tfidfw2v.shape[1],)),
  tf.keras.layers.Dense(128, activation='elu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(128, activation='elu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
# compile the keras model
dnn.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [ ]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

In [ ]:
history = dnn.fit(X_train_tfidfw2v, y_train, epochs=30, validation_data=(X_valid_tfidfw2v, y_valid), callbacks=[early_stopping_cb])

### Performance of DNN model

In [ ]:
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1) # set the vertical range to [0-1]
plt.show()

In [ ]:
dnn.evaluate(X_train_tfidfw2v, y_train, batch_size=128)

In [ ]:
dnn.evaluate(X_test_tfidfw2v, y_test, batch_size=128)

In [ ]:
#train predicted probability
train_proba = dnn.predict(X_train_tfidfw2v)

#test predicted probability
test_proba = dnn.predict(X_test_tfidfw2v)

In [ ]:
#predicted classes on train data
y_train_predict = train_proba >0.5

#predicted classes on test data
y_test_predict = test_proba>0.5

In [ ]:
#accuracy score on train data
accuracy_score(y_train,y_train_predict)

In [ ]:
#accuracy score on test data
accuracy_score(y_test,y_test_predict)

In [ ]:
#confusion_matrix on train data

#rows are actual label in ascending order i.e. 0 then 1 and cols are predicted labels
confusion_matrix(y_train, y_train_predict, normalize='true')

In [ ]:
#confusion_matrix on test data
#rows are actual label in ascending order i.e. 0 then 1 and cols are predicted labels
confusion_matrix(y_test, y_test_predict, normalize='true')

In [ ]:

#f1_score on the train data
print(f1_score(y_train, y_train_predict))

#f1_score on the test data
print(f1_score(y_test, y_test_predict))

**Observation: Look at above confusion matrix: approx 25% of +ve data predicted as -ve and 13% of -ve data predicted as +ve. And also overall performance is not very good. It is performing badly than the bow and tfidf vectorization.**

In [ ]:
#model saving path
model_version = "0001"
model_path = "/content/drive/MyDrive/applied ai/myself work/amazon fine food reviews/saved_models/dnn_tfidfw2v"
model_path = os.path.join(model_path, model_version)

**Saving trained model:**

In [ ]:
# dnn.save(model_path)

**Loading trained data:**

In [ ]:
new_model = tf.keras.models.load_model(model_path)

In [ ]:
new_model.summary()

#Conclusion:

**Logistic regression with the bow vectorization gives us the best performance with approx 91% train accuracy and approx 90% test accuracy.**

**The End**